In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%bash
pip install torch
pip install datasets
pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
from datasets import load_dataset
import pickle
from nltk.tokenize import word_tokenize
import nltk
import os
import random

nltk.download('punkt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
# Load dataset
dataset = load_dataset('rotten_tomatoes')
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

train, val, test = train_dataset.to_pandas(), validation_dataset.to_pandas(), test_dataset.to_pandas()
max_length = max(0, train['text'].apply(lambda x: len(x)).max())
max_length = max(max_length, val['text'].apply(lambda x: len(x)).max())
max_length = max(max_length, test['text'].apply(lambda x: len(x)).max())
max_length += 5

# Load pretrained embeddings
with open('drive/MyDrive/SC4002/embedding_matrix_new.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

# Load word-to-index mapping
with open('drive/MyDrive/SC4002/vocab_word_to_index_new.pkl', 'rb') as f:
    word_to_idx = pickle.load(f)

vocab_size, embedding_dim = embedding_matrix.shape
padding_embedding = np.zeros((50, embedding_dim))
embedding_matrix = np.vstack((embedding_matrix, padding_embedding))

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
# Tokenize data using NLTK
# def tokenize_data(dataset, word_to_idx, max_length = 100):
#     tokenized_texts = []
#     labels = []

#     for text in dataset['text']:
#         tokens = word_tokenize(text.lower())
#         indexed_tokens = [word_to_idx.get(word, len(word_to_idx) - 1) for word in tokens]
#         if len(indexed_tokens) > max_length:
#             indexed_tokens = indexed_tokens[:max_length]
#         else:
#             indexed_tokens += [len(word_to_idx)] * (max_length - len(indexed_tokens))  # Padding

#         tokenized_texts.append(indexed_tokens)

#     labels = dataset['label']
#     return torch.tensor(tokenized_texts), torch.tensor(labels)

# train = train_dataset.to_pandas()
# max_seq_len = max(0, train['text'].apply(lambda x: len(x)).max())
# train_inputs, train_labels = tokenize_data(train_dataset, word_to_idx, max_seq_len)
# val_inputs, val_labels = tokenize_data(validation_dataset, word_to_idx, max_seq_len)
# test_inputs, test_labels = tokenize_data(test_dataset, word_to_idx, max_seq_len)

class CustomDataset(Dataset):
    def __init__(self, texts, labels, word_to_idx):

        inputs = []
        lengths = []

        for text in texts:
            tokens = word_tokenize(text.lower())
            indexed_tokens = [word_to_idx.get(word, len(word_to_idx) - 1) for word in tokens]
            sequence_length = len(indexed_tokens)
            lengths.append(min(sequence_length, max_length))

            if sequence_length > max_length:
                indexed_tokens = indexed_tokens[:max_length]
            else:
                indexed_tokens += [len(word_to_idx)] * (max_length - sequence_length)  # Padding

            inputs.append(indexed_tokens)

        self.inputs = torch.tensor(inputs).to(device)
        self.labels = torch.tensor(labels).to(device)
        self.lengths = torch.tensor(lengths).to(device)

    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self,idx):
        return self.inputs[idx], self.labels[idx], self.lengths[idx]

# train_data = TensorDataset(train_inputs, train_labels)
# val_data = TensorDataset(val_inputs, val_labels)
# test_data = TensorDataset(test_inputs, test_labels)

train_data = CustomDataset(train_dataset['text'], train_dataset['label'], word_to_idx)
val_data = CustomDataset(validation_dataset['text'], validation_dataset['label'], word_to_idx)
test_data = CustomDataset(test_dataset['text'], test_dataset['label'], word_to_idx)

def worker_init_fn(worker_id):
    np.random.seed(42 + worker_id)
    random.seed(42 + worker_id)

batch_size = 32
train_loader = DataLoader(train_data, shuffle = True, batch_size = batch_size, worker_init_fn = worker_init_fn)
val_loader = DataLoader(val_data, batch_size = batch_size)
test_loader = DataLoader(test_data, batch_size = batch_size)

In [ ]:
c = 0
for batch in train_loader:
    c += 1
    print(batch)
print(f'{c} batches')

[tensor([[ 9398, 13878,  1458,  ..., 16536, 16536, 16536],
        [11444, 16535, 12983,  ..., 16536, 16536, 16536],
        [  203,  6361,  1472,  ..., 16536, 16536, 16536],
        ...,
        [ 6920,  8372,  9606,  ..., 16536, 16536, 16536],
        [ 4459, 13810,  9055,  ..., 16536, 16536, 16536],
        [ 5602, 11338,  4682,  ..., 16536, 16536, 16536]], device='cuda:0'), tensor([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
        0, 0, 1, 1, 1, 0, 0, 0], device='cuda:0'), tensor([22, 27, 19, 29, 19,  6, 27, 29, 32, 27, 14, 25, 44, 33, 18, 36,  4, 14,
        25, 36, 26, 29, 23, 23, 15, 24, 34, 12, 10,  9, 22, 17],
       device='cuda:0')]
[tensor([[12492, 13220,  4944,  ..., 16536, 16536, 16536],
        [11444, 15902,  5214,  ..., 16536, 16536, 16536],
        [12758,  8962,  9687,  ..., 16536, 16536, 16536],
        ...,
        [12563,  1472,  4909,  ..., 16536, 16536, 16536],
        [11444, 16338,  1472,  ..., 16536, 16536, 16536],
        [ 4877

In [ ]:
# Define Model with biLSTM and biGRU options
class SentimentModel(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, num_layers, output_size = 1, model_type = 'lstm'):
        super(SentimentModel, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype = torch.float32), freeze = False, padding_idx = len(word_to_idx))

        if model_type == 'lstm':
            self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers = num_layers, bidirectional = True, batch_first = True)
        elif model_type == 'gru':
            self.rnn = nn.GRU(embedding_dim, hidden_size, num_layers = num_layers, bidirectional = True, batch_first = True)

        self.fc = nn.Linear(hidden_size * 2, 1)  # 2 for bidirectional and 1 output class
        # self.softmax = nn.Softmax(dim = 1)
        # self.sigmoid = nn.Sigmoid()
        self.init_weights()

    def init_weights(self):
        # Initialize Embedding Layer
        nn.init.uniform_(self.embedding.weight, -0.01, 0.01)

        # Initialize RNN (LSTM/GRU) weights and biases
        for name, param in self.rnn.named_parameters():
            if 'weight_ih' in name:  # Input to hidden weights
                nn.init.xavier_uniform_(param.data)  # Xavier initialization
            elif 'weight_hh' in name:  # Hidden to hidden weights
                nn.init.orthogonal_(param.data)  # Orthogonal initialization
            elif 'bias' in name:
                nn.init.zeros_(param.data)  # Zero bias initialization

        # Initialize Linear (Fully connected) layer
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed_embedded = pack_padded_sequence(embedded, lengths.cpu(), batch_first = True, enforce_sorted = False)
        packed_rnn_out, _ = self.rnn(packed_embedded)
        rnn_out, _ = pad_packed_sequence(packed_rnn_out, batch_first = True)
        final_feature_map = rnn_out[torch.arange(rnn_out.size(0)), lengths - 1]
        final_out = self.fc(final_feature_map)
        return final_out

In [ ]:
# Training loop
def train_model(model, train_loader, val_loader, epochs, lr):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr = lr)

    best_val_acc = 0

    model = model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        scaler = torch.amp.GradScaler()

        for inputs, labels, lengths in train_loader:
            labels = labels.float().unsqueeze(1)
            optimizer.zero_grad()
            with torch.amp.autocast():
                outputs = model(inputs, lengths)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            # loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 4.0)
            # optimizer.step()
            running_loss += loss.item()

        _, val_acc = evaluate_model(model, val_loader)
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {(running_loss / len(train_loader)):.4f}, Val Accuracy: {val_acc:.4f}')

    print('Training complete')

# Evaluation function
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    # all_logits = []
    with torch.no_grad():
        for inputs, labels, lengths in loader:
            # inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs, lengths)
            # print(outputs)
            predictions = (outputs >= 0.5).float()
            # logits, predictions = torch.max(outputs, 1)
            all_predictions.extend(predictions.cpu().numpy())
            # all_logits.extend(logits.cpu().numpy())
            total += labels.size(0)
            correct += (predictions == labels.float().unsqueeze(1)).sum().item()

    accuracy = correct / total
    return all_predictions, accuracy

In [ ]:
# biGRU

hidden_size = 64
num_layers = 3
learning_rate = 0.01
epochs = 10

print('Training biGRU Model')
biGRU_model = SentimentModel(embedding_matrix, hidden_size, num_layers, model_type = 'gru')
train_model(biGRU_model, train_loader, val_loader, epochs, learning_rate)

print('Evaluating biGRU Model on Test Set')
predictions_biGRU, test_accuracy_biGRU = evaluate_model(biGRU_model, test_loader)
print(f'Test Accuracy biGRU: {test_accuracy_biGRU:.4f}')

Training biGRU Model


<ipython-input-9-f847bba5d443>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/10, Train Loss: 0.5940, Val Accuracy: 0.7458
Epoch 2/10, Train Loss: 0.2873, Val Accuracy: 0.7383
Epoch 3/10, Train Loss: 0.1209, Val Accuracy: 0.7326
Epoch 4/10, Train Loss: 0.0633, Val Accuracy: 0.7242
Epoch 5/10, Train Loss: 0.0413, Val Accuracy: 0.7167
Epoch 6/10, Train Loss: 0.0356, Val Accuracy: 0.7383
Epoch 7/10, Train Loss: 0.0352, Val Accuracy: 0.7261
Epoch 8/10, Train Loss: 0.0362, Val Accuracy: 0.7336
Epoch 9/10, Train Loss: 0.0319, Val Accuracy: 0.7280
Epoch 10/10, Train Loss: 0.0263, Val Accuracy: 0.7289
Training complete
Evaluating biGRU Model on Test Set
Test Accuracy biGRU: 0.7570


In [ ]:
# biLSTM

hidden_size = 64
num_layers = 3
learning_rate = 0.01
epochs = 10

print('Training biLSTM Model')
biLSTM_model = SentimentModel(embedding_matrix, hidden_size, num_layers, model_type = 'lstm')
train_model(biLSTM_model, train_loader, val_loader, epochs, learning_rate)

print('Evaluating biLSTM Model on Test Set')
predictions_biLSTM, test_accuracy_biLSTM = evaluate_model(biLSTM_model, test_loader)
print(f'Test Accuracy biLSTM: {test_accuracy_biLSTM:.4f}')

Training biLSTM Model


<ipython-input-9-f847bba5d443>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/10, Train Loss: 0.6652, Val Accuracy: 0.6398
Epoch 2/10, Train Loss: 0.4058, Val Accuracy: 0.7308
Epoch 3/10, Train Loss: 0.1937, Val Accuracy: 0.7467
Epoch 4/10, Train Loss: 0.1062, Val Accuracy: 0.7467
Epoch 5/10, Train Loss: 0.0637, Val Accuracy: 0.7477
Epoch 6/10, Train Loss: 0.0419, Val Accuracy: 0.7373
Epoch 7/10, Train Loss: 0.0318, Val Accuracy: 0.7392
Epoch 8/10, Train Loss: 0.0275, Val Accuracy: 0.7411
Epoch 9/10, Train Loss: 0.0239, Val Accuracy: 0.7392
Epoch 10/10, Train Loss: 0.0235, Val Accuracy: 0.7317
Training complete
Evaluating biLSTM Model on Test Set
Test Accuracy biLSTM: 0.7458
